# Syncronization of Microscope frame to camera frames


Loading the Camera csv(video_timestamp) and converting into a data frame in milliseconds


In [34]:
#Importing the packages

import pandas as pd
import numpy as np

In [35]:
# Load the data files
camera_df = pd.read_csv('camera.csv',sep=r'\s+', names= ['hrs','min','sec','mill'])
microscope_df = pd.read_csv('sync_2019-10-01T13_28_00 - Copy.csv',sep=r'\s+',names= ['hrs','min','sec','mill','frame','frame_no'])

In [36]:
camera_df.head(2)

,hrs,min,sec,mill
0,13,28,0,457
1,13,28,0,464


In [37]:
camera_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   hrs     55151 non-null  int64
 1   min     55151 non-null  int64
 2   sec     55151 non-null  int64
 3   mill    55151 non-null  int64
dtypes: int64(4)
memory usage: 1.7 MB


In [38]:
microscope_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27011 entries, 0 to 27010
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hrs       27011 non-null  int64 
 1   min       27011 non-null  int64 
 2   sec       27011 non-null  int64 
 3   mill      27011 non-null  int64 
 4   frame     27011 non-null  object
 5   frame_no  27011 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ MB


In [39]:
microscope_df.head(2)

,hrs,min,sec,mill,frame,frame_no
0,13,28,9,867,"Fra,",2404444129
1,13,28,9,899,"Fra,",2404444163


In [40]:
# Convert time hr:min:sec:mill to milliseconds and creating a new column with millisec

def convert_to_ms(df):
   
    """ Convert hrs, min, sec, mill columns to total milliseconds and create a new column 'time_ms'. """
    return (df['hrs']*3600000 + df['min']*60000 + df['sec']*1000 + df['mill'])
   

In [41]:
camera_df['camera_time_in_ms'] = convert_to_ms(camera_df)
microscope_df['microscope_time_in_ms'] = convert_to_ms(microscope_df)

In [42]:
camera_df.head(2)

,hrs,min,sec,mill,camera_time_in_ms
0,13,28,0,457,48480457
1,13,28,0,464,48480464


In [43]:
camera_df.isnull().sum()

hrs                  0
min                  0
sec                  0
mill                 0
camera_time_in_ms    0
dtype: int64

In [44]:
microscope_df.isnull().sum()

hrs                      0
min                      0
sec                      0
mill                     0
frame                    0
frame_no                 0
microscope_time_in_ms    0
dtype: int64

In [45]:
camera_df[camera_df.duplicated(subset=['hrs','min','sec','mill','camera_time_in_ms'])]

,hrs,min,sec,mill,camera_time_in_ms


In [46]:
microscope_df[microscope_df.duplicated(subset=['hrs','min','sec','mill','frame','frame_no','microscope_time_in_ms'])]

,hrs,min,sec,mill,frame,frame_no,microscope_time_in_ms


# Observations and insights

No dupicates found in two files and no null values were found

Deleting the hr, min, sec, mill columns and keeping only millisecond column

In [47]:
camera_df.drop(columns=['hrs','min','sec','mill'], inplace =True)

In [48]:
microscope_df.drop(columns=['hrs','min','sec','mill','frame'],inplace=True)

In [49]:
microscope_df.head(3)

,frame_no,microscope_time_in_ms
0,2404444129,48489867
1,2404444163,48489899
2,2404444195,48489931


In [50]:
# Reordering the columns in micorscope_df

microscope_df = microscope_df[['microscope_time_in_ms','frame_no']]

In [51]:
microscope_df.head(3)

,microscope_time_in_ms,frame_no
0,48489867,2404444129
1,48489899,2404444163
2,48489931,2404444195


In [52]:
camera_df.head()

,camera_time_in_ms
0,48480457
1,48480464
2,48480479
3,48480496
4,48480512


In [53]:
microscope_df.tail()

,microscope_time_in_ms,frame_no
27006,49390531,2405344747
27007,49390564,2405344780
27008,49390596,2405344814
27009,49390633,2405344846
27010,49390666,2405344880


In [54]:
camera_df.tail()

,camera_time_in_ms
55146,49399783
55147,49399800
55148,49399817
55149,49399834
55150,49399850


In [55]:
# Ensuring both were sorted by time

camera_df = camera_df.sort_values('camera_time_in_ms').reset_index(drop=True)
microscope_df = microscope_df.sort_values('microscope_time_in_ms').reset_index(drop = True)

In [56]:


# --- Synchronize many camera frames to one microscope frame ---

def synchronize_many_to_one(camera_df, microscope_df, direction='camera_first'):
    """
    Synchronize multiple camera frames to each microscope frame.
    
    direction:
        'camera_first' → camera leads, assigns camera frames that occur *before or equal* to each microscope frame.
        'camera_after' → camera lags, assigns camera frames that occur *after or equal* to each microscope frame.
    
    Returns:
        result_df: DataFrame mapping each microscope frame to one or more camera timestamps.
        unused_cameras: DataFrame of camera frames that were not mapped to any microscope frame.
    """
    camera_used = set()
    sync_data = []

    cam_times = camera_df['camera_time_in_ms'].to_numpy()
    micro_times = microscope_df['microscope_time_in_ms'].to_numpy()

    for i, micro_time in enumerate(micro_times):
        if direction == 'camera_first':
            # Include all camera frames between previous microscope and current microscope
            lower_bound = micro_times[i-1] if i > 0 else -np.inf
            cams_in_range = camera_df[(camera_df['camera_time_in_ms'] > lower_bound) &
                                      (camera_df['camera_time_in_ms'] <= micro_time)]
        else:
            # Include all camera frames between current microscope and next microscope
            upper_bound = micro_times[i+1] if i < len(micro_times)-1 else np.inf
            cams_in_range = camera_df[(camera_df['camera_time_in_ms'] >= micro_time) &
                                      (camera_df['camera_time_in_ms'] < upper_bound)]

        # Mark camera frames as used
        camera_used.update(cams_in_range.index.tolist())

        # Save mapping (microscope frame → all matching camera frames)
        sync_data.append({
            'microscope_frame_no': microscope_df.loc[i, 'frame_no'],
            'microscope_time_in_ms': micro_time,
            'aligned_camera_times': cams_in_range['camera_time_in_ms'].tolist(),
            'num_camera_frames': len(cams_in_range)
        })

    # Converting to DataFrame
    result_df = pd.DataFrame(sync_data)

    # Unused camera frames
    unused_cameras = camera_df.drop(index=list(camera_used)).reset_index(drop=True)

    return result_df, unused_cameras








In [57]:
# --- Perform synchronization ---
final_synced_df, unused_camera_frames = synchronize_many_to_one(camera_df, microscope_df, direction='camera_first')


In [58]:
final_synced_df.head()

,microscope_frame_no,microscope_time_in_ms,aligned_camera_times,num_camera_frames
0,2404444129,48489867,"[48480457, 48480464, 48480479, 48480496, 48480...",566
1,2404444163,48489899,"[48489881, 48489898]",2
2,2404444195,48489931,"[48489915, 48489931]",2
3,2404444229,48489964,"[48489948, 48489964]",2
4,2404444262,48490001,"[48489981, 48489998]",2


In [59]:
final_synced_df.tail()

,microscope_frame_no,microscope_time_in_ms,aligned_camera_times,num_camera_frames
27006,2405344747,49390531,"[49390514, 49390531]",2
27007,2405344780,49390564,"[49390548, 49390564]",2
27008,2405344814,49390596,[49390581],1
27009,2405344846,49390633,"[49390598, 49390614, 49390631]",3
27010,2405344880,49390666,"[49390648, 49390665]",2


In [60]:
len(unused_camera_frames)


551

# Observations and insights
1. Syncronized and camera time frame to microscope with one to many mapping
2. To decrease the jetter effect choosing the one direction to map the camera time frames
3. Kept track of Unused camara frames in seperate data frame

Concerns :(

a single microscope time frame was mapped to list of camera timeframes
for 48489867(microscope ms) 516 values of camera frames were mapped